### Task 3: Forecast Future Market Trends (TSLA)

Use the trained model from Task 2 (ARIMA/SARIMA/LSTM) to generate 6–12 month forecasts, visualize with confidence intervals, and interpret trends and risks.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.data_manager import DataManager
from models.forecasting_models import ARIMAForecaster, SARIMAForecaster, LSTMForecaster
from src.model_selection import chronological_split, arima_order_grid_search
from src.forecasting_analysis import ForecastAnalyzer

plt.style.use('seaborn-v0_8')
%matplotlib inline


### 1) Load TSLA and refit a model from Task 2 (if needed)

For reproducibility, we reselect an ARIMA order via a quick grid search and fit on the full history.


In [ ]:
START_DATE = '2015-07-01'
END_DATE = '2025-07-31'
TICKER = ['TSLA']

dm = DataManager(data_source='yfinance')
raw = dm.fetch_data(TICKER, start_date=START_DATE, end_date=END_DATE, frequency='1d')
tsla_close = raw['TSLA']['Adj Close'].rename('TSLA').dropna()
tsla_close.tail()


In [ ]:
# Choose a model: 'ARIMA' (default here), or adapt to 'SARIMA'/'LSTM'
model_choice = 'ARIMA'

if model_choice == 'ARIMA':
    order, _ = arima_order_grid_search(tsla_close, p_values=range(0,4), d_values=range(0,2), q_values=range(0,4))
    model = ARIMAForecaster(order=order).fit(tsla_close)
elif model_choice == 'SARIMA':
    model = SARIMAForecaster(order=(1,1,1), seasonal_order=(1,1,1,12)).fit(tsla_close)
elif model_choice == 'LSTM':
    model = LSTMForecaster(units=64, epochs=10, batch_size=32).fit(tsla_close)
else:
    raise ValueError('Unsupported model choice')

model.name, getattr(model, 'order', None)


### 2) Generate 6-month and 12-month forecasts with confidence intervals


In [ ]:
analyzer = ForecastAnalyzer(history=tsla_close)
analyzer.attach_model(model.name, model)

fc_6m = analyzer.forecast(horizon_days=126, include_intervals=True, ci_alpha=0.1)   # ~6 months
fc_12m = analyzer.forecast(horizon_days=252, include_intervals=True, ci_alpha=0.1)  # ~12 months

(fc_6m.model_name, len(fc_6m.mean), len(fc_12m.mean))


### 3) Visualize forecasts against history


In [ ]:
fig, ax = plt.subplots(figsize=(14,5))

# Plot history (last 2 years for focus)
history_tail = tsla_close.tail(504)
history_tail.plot(ax=ax, label='History', color='black')

# Plot 6-month forecast
fc_6m.mean.plot(ax=ax, label='Forecast 6m')
if fc_6m.lower is not None and fc_6m.upper is not None:
    ax.fill_between(fc_6m.mean.index, fc_6m.lower, fc_6m.upper, color='C0', alpha=0.2)

# Plot 12-month forecast
fc_12m.mean.plot(ax=ax, label='Forecast 12m')
if fc_12m.lower is not None and fc_12m.upper is not None:
    ax.fill_between(fc_12m.mean.index, fc_12m.lower, fc_12m.upper, color='C1', alpha=0.2)

ax.set_title('TSLA: Future Forecasts (6m and 12m) with Confidence Intervals')
ax.legend();


### 4) Interpret the Results

- Trend Analysis: Discuss whether the central forecast suggests upward, downward, or stable movement.
- Patterns/Anomalies: Note any visible seasonality, mean-reversion, or outliers.
- Volatility and Risk: Examine the confidence intervals—do they widen with horizon? What does that say about uncertainty over longer forecasts?
- Market Opportunities/Risks: Identify potential opportunities (e.g., expected increases) and risks (e.g., high volatility, drawdowns).
